# Programming Assignment 2
IEMS 450-2, Zixin Zhang

In [1]:
import numpy as np


## Solver

### Solver options

In [2]:
'''
Options for the active-set QP solver
'''


class solverOptions:
    def __init__(self):
        self.print_level = "Verbose"
        self.step_tol = 1e-6
        self.term_tol = 1e-6
        self.feas_tol = 1e-9
        self.max_iter = 1000


### Active-set QP solver

In [3]:
'''
Active-set QP solver
'''


class Solver:
    def __init__(self, solver_options):
        self.solver_options = solver_options

    def print_final_status(self, status):
        match status:
            case 0:
                print("A KKT point is found!")
            case 1:
                print("Maximum number of iterations reached.")

    def active_set_solve(self, G, c, A, b, x0, W0):
        '''
        This function solves a QP with inequality constraints using the active-set method

        A QP is defined as follows:
        min  q(x) = 0.5*x'*G*x + c'*x
        s.t. A*x >= b

        Parameters:
            G:  n x n matrix
            c:  n-dim vector
            A:  m x n matrix
            b:  m-dim vector
            x0: starting point
            W0: initial working set, m-dim boolean vector, and a component is set to true if
                the corresponding constraint is in the active set

        Returns:
            x_sol:      solution, n-dim vector
            lambda_sol: corresponding optimal Lagrange multipliers, m-dim vector
            W_sol:      final working set, m-dim boolean vector
            status:     exit status

        Note:
            x0 should be feasible, and W0 should be a subset of the active constraints at x0
        '''
        xk = np.copy(x0)
        Wk = np.copy(W0)
        num_iter = 0

        output_header = '%5s %15s %20s %15s %15s %20s %20s'% \
                        ('iter', 'f_val', 'primal_feas', 'alpha', 'max{|p_k|}', 'size_working_set', 'change_cons_idx')
        
        fk = 0.0
        alpha = 0
        pk = np.zeros(np.shape(xk))
        lambda_sol = np.zeros((np.size(np.where(Wk == True)), 1))
        change_cons_idx = 0
        size_Wk = np.size(np.where(Wk == True))
        while True:
            # Evaluate the objective function
            fk = 0.5 * xk.T @ G @ xk + c.T @ xk

            # Evaluate the primal feasibility
            cons_k = A @ xk - b
            primal_feas = 0.0
            tmp_1 = np.where(cons_k < 0)[0]
            if np.size(tmp_1) > 0:
                tmp_2 = np.hstack((0, cons_k[tmp_1[0]]))
                primal_feas = np.linalg.norm(tmp_2, np.inf)

            # Print the iteration information
            if self.solver_options.print_level == "Verbose":
                # Print header every 10 iterstion
                if num_iter % 10 == 0:
                    print(output_header)
                output_line = '%5d %15E %20E %15E %15E %20d %20s'% \
                              (num_iter, fk[0][0], primal_feas, alpha, np.linalg.norm(pk, np.inf), size_Wk, change_cons_idx)
                change_cons_idx = 0
                print(output_line)

            # Initialize active constraints matrix
            A_active = np.copy(A[Wk, :])
            b_active = np.copy(b[Wk])

            # Solve the QP with the current working set
            x_sol, lambda_sol, status = self.solve_eq_qp(G, c, A_active, b_active, xk)
            pk = x_sol - xk
            # If xk is a minimizer with Wk
            if np.linalg.norm(pk, np.inf) < self.solver_options.step_tol:
                alpha = 0
                # If the KKT conditions are satisfied, return
                if np.all(lambda_sol >= -self.solver_options.term_tol):
                    status = 0
                    if self.solver_options.print_level == "Verbose":
                        # fk = 0.5 * xk.T @ G @ xk + c.T @ xk
                        output_line = '%5d %15E %20E %15E %15E %20d %20s'% \
                                      (num_iter+1, fk[0][0], primal_feas, alpha, np.linalg.norm(pk, np.inf), size_Wk, change_cons_idx)
                        change_cons_idx = 0
                        print(output_line)
                    if self.solver_options.print_level == "Verbose" or self.solver_options.print_level == "Final":
                        # Evaluate the dual feasibility
                        active_cons_idx = np.where(Wk == True)[0]
                        dual_feas = 0.0
                        if np.size(active_cons_idx) > 0:
                            dual_feas = G @ xk + c - A[active_cons_idx, :].T @ lambda_sol
                        else:
                            dual_feas = G @ xk + c

                        # Print the final status
                        self.print_final_status(status)
                        print("Number of iterations: ", num_iter)
                        print("Size of the working set: ", size_Wk)
                        print("Optimal objective function value: ", fk[0][0])
                        print("Primal feasibility: ", primal_feas)
                        print("Dual feasibility: ", np.linalg.norm(dual_feas, np.inf))
                    return xk, lambda_sol, Wk, status
                # Otherwise, remove the constraints with the most negative lambda
                else:
                    idx = np.argmin(lambda_sol)
                    true_indices = np.where(Wk)[0]
                    if idx < len(true_indices):
                        Wk[true_indices[idx]] = False
                        size_Wk = np.size(np.where(Wk == True))
                        change_cons_idx = -(true_indices[idx] + 1)

            # If xk is not a minimizer with Wk
            else:
                # Find the step length
                tmp = np.inf
                for i in range(len(Wk)):
                    if Wk[i] == False and np.dot(A[i, :], pk) < 0:
                        tmp = min(
                            tmp, (b[i] - np.dot(A[i, :], xk)) / np.dot(A[i, :], pk))
                alpha = min(1, tmp)
                if alpha != 1:
                    alpha = alpha[0]

                # Update xk
                xk = xk + alpha * pk

                # Check if there are blocking constraints
                for i in range(len(Wk)):
                    if Wk[i] == False and abs(np.dot(A[i, :], xk) - b[i]) <= self.solver_options.feas_tol:
                        Wk[i] = True
                        size_Wk = np.size(np.where(Wk == True))
                        change_cons_idx = i + 1
                        break

            # Check if the maximum number of iterations is reached
            num_iter += 1
            if num_iter >= self.solver_options.max_iter:
                status = 1
                if self.solver_options.print_level == "Verbose":
                    fk = 0.5 * xk.T @ G @ xk + c.T @ xk
                    output_line = '%5d %15E %20E %15E %15E %20d %20s'% \
                                    (num_iter+1, fk[0][0], primal_feas, alpha, np.linalg.norm(pk, np.inf), size_Wk, change_cons_idx)
                    change_cons_idx = 0
                    print(output_line)
                if self.solver_options.print_level == "Verbose" or self.solver_options.print_level == "Final":
                    # Evaluate the dual feasibility
                    active_cons_idx = np.where(Wk == True)[0]
                    dual_feas = 0.0
                    if np.size(active_cons_idx) > 0:
                        dual_feas = G @ xk + c - A[active_cons_idx, :].T @ lambda_sol
                    else:
                        dual_feas = G @ xk + c
                    fk = 0.5 * xk.T @ G @ xk + c.T @ xk
                    self.print_final_status(status)
                    print("Number of iterations: ", num_iter)
                    print("Size of the working set: ", size_Wk)
                    print("Optimal objective function value: ", fk[0][0])
                    print("Primal feasibility: ", primal_feas)
                    print("Dual feasibility: ", np.linalg.norm(dual_feas, np.inf))
                return xk, lambda_sol, Wk, status

    def solve_eq_qp(self, G, c, A, b, x0):
        '''
        This function solves a QP with equality constraints using Schur-complement method

        A QP is defined as follows:
        min  q(x) = 0.5*x'*G*x + c'*x
        s.t. A*x = b

        Parameters:
            G:  n x n matrix
            c:  n-dim vector
            A:  m x n matrix
            b:  m-dim vector
            x0: starting point

        Returns:
            x_sol:       solution, n-dim vector
            lambda_sol:  corresponding optimal Lagrange multipliers, m-dim vector
            status: exit status
        '''
        n = np.shape(x0)[0]
        G_inv = np.linalg.inv(G)
        tmp_inv = np.linalg.inv((A @ G_inv @ A.T))
        C = G_inv - G_inv @ A.T @ tmp_inv @ A @ G_inv
        E = G_inv @ A.T @ tmp_inv
        F = -tmp_inv
        g = c + G @ x0
        h = A @ x0 - b
        K_inv_top = np.hstack((C, E))
        K_inv_bottom = np.hstack((E.T, F))
        K_inv = np.vstack((K_inv_top, K_inv_bottom))
        gh_vec = np.vstack((g, h))
        sol_vec = K_inv @ gh_vec
        p = -sol_vec[:n, :]
        lambda_sol = sol_vec[n:, :]
        x_sol = x0 + p
        status = 0
        return x_sol, lambda_sol, status


## Example 16.4

In [4]:
# min  q(x) = (x1-1)^2 + (x2-2.5)^2
# s.t. x1 - 2*x2 >= -2
#      -x1 - 2*x2 >= -6
#      -x1 + 2*x2 >= -2
#      x1 >= 0
#      x2 >= 0
solver_options = solverOptions()
solver = Solver(solver_options)
G = np.array([[2, 0], [0, 2]])
c = np.array([[-2], [-5]])
A = np.array([[1, -2], [-1, -2], [-1, 2], [1, 0], [0, 1]])
b = np.array([[-2], [-6], [-2], [0], [0]])


### $x_0 = (2, 0)$ and $W_0 = \emptyset$

In [5]:
x0 = np.array([[2], [0]])
W0 = np.array([False, False, False, False, False]) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1   -6.444444E+00         0.000000E+00    6.666667E-01    2.500000E+00                    1                    1
    2   -6.450000E+00         0.000000E+00    1.000000E+00    6.666667E-02                    1                    0
    3   -6.450000E+00         0.000000E+00    0.000000E+00    2.220446E-16                    1                    0
A KKT point is found!
Number of iterations:  2
Size of the working set:  1
Optimal objective function value:  -6.450000000000001
Primal feasibility:  0.0
Dual feasibility:  2.220446049250313e-16
x_sol:  [1.4 1.7]
lambda_sol:  [0.8]
W_sol:  [1]


The solver reached the active constraint immediately and found a KKT point.

### $x_0 = (2, 0)$ and $W_0 = {3, 5}$

In [6]:
x0 = np.array([[2], [0]])
W0 = np.array([False, False, True, False, True]) # 3, 5
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    2                    0
    1    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    1                   -3
    2   -1.000000E+00         0.000000E+00    1.000000E+00    1.000000E+00                    1                    0
    3   -1.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                   -5
    4   -6.250000E+00         0.000000E+00    6.000000E-01    2.500000E+00                    1                    1
    5   -6.450000E+00         0.000000E+00    1.000000E+00    4.000000E-01                    1                    0
    6   -6.450000E+00         0.000000E+00    0.000000E+00    2.220446E-16                    1                    0
A KKT point is found!
Number of iterations:  5
Size of the worki

The solver took a few iterations to reach the real active constraint and then found the KKT point.

### $x_0 = (2, 0)$ and $W_0 = {3}$

In [7]:
x0 = np.array([[2], [0]])
W0 = np.array([False, False, True, False, False]) # 3
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    1                    0
    1   -5.000000E-02         4.440892E-16    1.000000E+00    2.000000E-01                    1                    0
    2   -5.000000E-02         4.440892E-16    0.000000E+00    5.551115E-17                    0                   -3
    3   -6.450000E+00         0.000000E+00    6.666667E-01    2.400000E+00                    1                    1
    4   -6.450000E+00         0.000000E+00    0.000000E+00    4.440892E-16                    1                    0
A KKT point is found!
Number of iterations:  3
Size of the working set:  1
Optimal objective function value:  -6.449999999999998
Primal feasibility:  0.0
Dual feasibility:  6.661338147750939e-16
x_sol:  [1.4 1.7]
lambda_sol:  [0.8]
W_sol:  [1]


The solver took a few iterations to reach the real active constraint and then found the KKT point.

### $x_0 = (2, 0)$ and $W_0 = {5}$

In [8]:
x0 = np.array([[2], [0]])
W0 = np.array([False, False, False, False, True]) # 5
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    1                    0
    1   -1.000000E+00         0.000000E+00    1.000000E+00    1.000000E+00                    1                    0
    2   -1.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                   -5
    3   -6.250000E+00         0.000000E+00    6.000000E-01    2.500000E+00                    1                    1
    4   -6.450000E+00         0.000000E+00    1.000000E+00    4.000000E-01                    1                    0
    5   -6.450000E+00         0.000000E+00    0.000000E+00    2.220446E-16                    1                    0
A KKT point is found!
Number of iterations:  4
Size of the working set:  1
Optimal objective function value:  -6.450000000000001
Primal feasibility:  0.0
Dual feasibility:  2.220446

The solver took a few iterations to reach the real active constraint and then found the KKT point.

We can find that, $W_0=\emptyset$ has the fastest convergence. This is because an incorrect initial working set can lead to the solver being incorrectly constrained, so it will take more time to find the correct active constraints. In contrast, starting directly from the unconstrained EQP gives the solver more "freedom" to explore and find the correct activation constraints quickly.

## Problem 16.17

In [9]:
# min  q(x) = x1^2 + x2^2 - 6*x1 - 4*x2
# s.t. -x1 - x2 >= -3
#      x1 >= 0
#      x2 >= 0
solver_options = solverOptions()
solver = Solver(solver_options)
G = np.array([[2, 0], [0, 2]])
c = np.array([[-6], [-4]])
A = np.array([[-1, -1], [1, 0], [0, 1]])
b = np.array([[-3], [0], [0]])


### $x_0 = (0, 0)$ and $W_0 = \emptyset$

In [10]:
x0 = np.array([[0], [0]])
W0 = np.array([False, False, False]) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1   -1.092000E+01         0.000000E+00    6.000000E-01    3.000000E+00                    1                    1
    2   -1.100000E+01         0.000000E+00    1.000000E+00    2.000000E-01                    1                    0
    3   -1.100000E+01         0.000000E+00    0.000000E+00    0.000000E+00                    1                    0
A KKT point is found!
Number of iterations:  2
Size of the working set:  1
Optimal objective function value:  -11.0
Primal feasibility:  0.0
Dual feasibility:  0.0
x_sol:  [2. 1.]
lambda_sol:  [2.]
W_sol:  [1]


The solver reached the active constraint immediately and found a KKT point.

### $x_0 = (0, 0)$ and $W_0 = {2, 3}$

In [11]:
x0 = np.array([[0], [0]])
W0 = np.array([False, True, True]) # 2, 3
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    2                    0
    1    0.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    1                   -2
    2   -9.000000E+00         0.000000E+00    1.000000E+00    3.000000E+00                    2                    1
    3   -9.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    1                   -3
    4   -1.100000E+01         0.000000E+00    1.000000E+00    1.000000E+00                    1                    0
    5   -1.100000E+01         0.000000E+00    0.000000E+00    0.000000E+00                    1                    0
A KKT point is found!
Number of iterations:  4
Size of the working set:  1
Optimal objective function value:  -11.0
Primal feasibility:  0.0
Dual feasibility:  0.0
x_sol:  [2. 1.]
l

The solver took a few iterations to reach the real active constraint and then found the KKT point.

### $x_0 = (1, 1)$ and $W_0 = \emptyset$

In [12]:
x0 = np.array([[1], [1]])
W0 = np.array([False, False, False]) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0   -8.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1   -1.077778E+01         0.000000E+00    3.333333E-01    2.000000E+00                    1                    1
    2   -1.100000E+01         0.000000E+00    1.000000E+00    3.333333E-01                    1                    0
    3   -1.100000E+01         0.000000E+00    0.000000E+00    0.000000E+00                    1                    0
A KKT point is found!
Number of iterations:  2
Size of the working set:  1
Optimal objective function value:  -11.0
Primal feasibility:  0.0
Dual feasibility:  0.0
x_sol:  [2. 1.]
lambda_sol:  [2.]
W_sol:  [1]


The solver reached the active constraint immediately and found a KKT point.

Something similar to what we saw in Example 16.4 happens again. The solver converged faster when the initial working set is empty.

## Problem 16.9

In [13]:
# min  q(x) = 3*x1^2 + 2*x1*x2 + x1*x3 + 2.5*x2^2 + 2*x2*x3 + 2*x3^2 - 8*x1 - 3*x2 - 3*x3
# s.t. -x1 - x2 >= -3.0001
#      x1 + x2 >= 3
#      -x2 - x3 >= -0.0001
#      x2 + x3 >= 0
solver_options = solverOptions()
solver = Solver(solver_options)
G = np.array([[6, 2, 1], [2, 5, 2], [1, 2, 4]])
c = np.array([[-8], [-3], [-3]])
A = np.array([[-1, 0, -1], [1, 0, 1], [0, -1, -1], [0, 1, 1]])
b = np.array([[-3.0001], [3], [-0.0001], [0]])

x0 = np.array([[3], [0], [0]])
W0 = np.array([False, False, False, False])

x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol: ", x_sol.flatten())
print("lambda_sol: ", lambda_sol.flatten())
print("W_sol: ", np.where(W_sol == True)[0] + 1)


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    3.000000E+00         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    3.000000E+00         0.000000E+00   -0.000000E+00    1.710843E+00                    1                    2
    2    2.997800E+00         1.355253E-20    1.538462E-04    1.250000E+00                    2                    3
    3   -3.500200E+00         0.000000E+00    1.000000E+00    9.998462E-01                    2                    0
    4   -3.500200E+00         0.000000E+00    0.000000E+00    2.220446E-16                    2                    0
A KKT point is found!
Number of iterations:  3
Size of the working set:  2
Optimal objective function value:  -3.5001999846153833
Primal feasibility:  0.0
Dual feasibility:  1.1102230246251565e-15
x_sol:  [ 1.99996154 -0.99993846  1.00003846]
lambda_sol:  [2.99993077 1.99969231]
W_sol:  [2 3]


The solver converges to a point where constraint 2 and 3 are active, so the final solution breaks the second equality constraint of the original EQP.

## Random QPs

### $n = 10, m = 10$

In [14]:
np.random.seed(0)
n = 10
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    4.127190E+01         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    3.997363E+01         8.881784E-16    9.915527E-04    2.653981E+02                    1                    9
    2    3.993690E+01         1.776357E-15    8.256863E-05    7.038143E+01                    2                    7
    3    3.885808E+01         8.881784E-16    4.233759E-03    2.232090E+01                    3                    5
    4    3.402546E+01         6.661338E-16    2.054243E-02    1.834758E+01                    4                    2
    5    2.862408E+01         8.881784E-15    2.477238E-02    1.462352E+01                    5                   10
    6   -4.600730E+01         8.251178E-13    1.000000E+00    1.304440E+01                    5                    0
    7   -4.600730E+01         8.251178E-13    0.000000E+00    1.

The solver took a few iterations to add blocking constraints into the working set. Then it reached the real active constraints and find the solution.

### $n = 100, m = 10$

In [15]:
np.random.seed(0)
n = 100
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    3.101770E+04         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    3.101485E+04         0.000000E+00    4.234077E-05    6.793431E+02                    1                    1
    2    3.098192E+04         6.643575E-13    5.192589E-04    1.859900E+02                    2                    6
    3    3.097112E+04         1.389111E-12    1.706682E-04    1.423827E+02                    3                    4
    4    3.096734E+04         1.492140E-12    6.011212E-05    6.144080E+01                    4                    2
    5    3.055663E+04         2.195222E-11    6.593916E-03    2.654821E+01                    5                    7
    6   -2.620730E+02         2.108575E-09    1.000000E+00    2.044970E+01                    5                    0
    7   -2.620730E+02         2.108575E-09    0.000000E+00    4.

The solver took a few iterations to add blocking constraints into the working set. Then it reached the real active constraints and find the solution.

### $n = 10, m = 100$

In [16]:
np.random.seed(0)
n = 10
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    2.332498E+01         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    2.290735E+01         0.000000E+00    3.278403E-04    2.661451E+02                    1                   42
    2    2.266906E+01         0.000000E+00    1.631511E-03    1.139225E+01                    2                   99
    3    2.224750E+01         0.000000E+00    3.639649E-03    1.195929E+01                    3                   94
    4    2.219083E+01         4.440892E-16    5.939505E-04    8.547992E+00                    4                   93
    5    2.201751E+01         8.881784E-16    2.289645E-03    6.724009E+00                    5                   16
    6    2.150021E+01         0.000000E+00    6.970168E-03    5.638954E+00                    6                   90
    7    2.036360E+01         0.000000E+00    3.501081E-02    1.

The solver took a few iterations to add blocking constraints into the working set. Then it reached the real active constraints and find the solution.

We can find that the number of constraints in the working set jumped back and forth for a few times when the solver was approaching the optimal solution. Since $m>>n$ in this QP, it is very easy for the solver to reach a blocking constraint. The solver found a blocking constraint, added it to the working set, solved a new EQP, found negative multipliers, throwed away a constraint out of the working set, solved a new EQP, moved forward, got blocked... This process repeated happened a few times because there are too many constraints.

### $n = 100, m = 200$

In [17]:
np.random.seed(0)
n = 100
m = 200
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    2.509339E+04         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    2.509111E+04         8.881784E-16    4.114860E-05    6.786783E+02                    1                   79
    2    2.509067E+04         3.552714E-15    8.378208E-06    2.337548E+02                    2                    1
    3    2.508283E+04         5.666578E-13    1.526248E-04    1.136684E+02                    3                   32
    4    2.508199E+04         6.057377E-13    1.634314E-05    8.564333E+01                    4                  166
    5    2.506420E+04         1.426415E-12    3.481346E-04    6.083368E+01                    5                  197
    6    2.502394E+04         3.252509E-12    7.958683E-04    1.075631E+01                    6                   86
    7    2.500754E+04         1.143974E-12    3.247620E-04    1.

The solver took 100 iterations to add blocking constraints into the working set. Then it reached the real active constraints and find the solution.

A similar "back and forth" change in the size of the working set occurs due to $m>>n$.

### $n = 200, m = 100$

In [18]:
np.random.seed(0)
n = 200
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    2.258322E+05         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    2.258059E+05         3.552714E-15    5.749223E-05    5.941126E+02                    1                   26
    2    2.257790E+05         1.776357E-15    5.907350E-05    3.338526E+02                    2                    4
    3    2.257699E+05         1.101341E-13    2.006906E-05    8.768567E+01                    3                   55
    4    2.257232E+05         5.364598E-13    1.028810E-04    6.639022E+01                    4                   28
    5    2.256883E+05         5.684342E-14    7.703785E-05    4.949059E+01                    5                   29
    6    2.256230E+05         2.479794E-12    1.441859E-04    4.384098E+01                    6                   94
    7    2.255307E+05         4.090950E-12    2.041318E-04    3.

The solver found a KKT point. The size of the working set increased to 65 first, and then decreased to 54. 

We can find that the "back and forth" change doesn't happen here, because $n>>m$. First, the solver has always been blocked by new constraints in the forward direction, so the size of the working set has been growing. It was trying to "explore" the space and find all possible constraints. After that, as solver got close to the optimal solution, it started to throw away unnecessay constraints. This phenomenon does not occur for $n<<m$, because such a QP is "overconstrained", the feasible region is more "determinstic" and the solver doesn't need to do exploration.

### $n = 1000, m = 1000$

In [19]:
np.random.seed(0)
n = 1000
m = 1000
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 10*(0.5-np.random.rand(n,1))
A = 10*(0.5-np.random.rand(m,n))
b = 10*(0.5-np.random.rand(m,1))
x_feas = np.random.rand(n,1)
b = A @ x_feas - 1

x0 = x_feas
W0 = np.array([False]*m) # empty
x_sol, lambda_sol, W_sol, status = solver.active_set_solve(G, c, A, b, x0, W0)
print("x_sol[:10]: ", x_sol.flatten()[:10])
print("lambda_sol[:10]: ", lambda_sol.flatten()[:10])
print("W_sol[:10]: ", (np.where(W_sol == True)[0] + 1)[:10])


 iter           f_val          primal_feas           alpha      max{|p_k|}     size_working_set      change_cons_idx
    0    3.109524E+07         0.000000E+00    0.000000E+00    0.000000E+00                    0                    0
    1    3.109512E+07         1.776357E-14    1.944834E-06    6.741213E+03                    1                  212
    2    3.109494E+07         1.659828E-11    2.757843E-06    1.100897E+03                    2                  356
    3    3.109479E+07         1.356071E-11    2.529049E-06    8.251849E+02                    3                  575
    4    3.109461E+07         1.153566E-11    2.771829E-06    4.486593E+02                    4                  269
    5    3.109442E+07         6.572876E-11    3.031661E-06    3.530598E+02                    5                  713
    6    3.109433E+07         1.025953E-10    1.538405E-06    3.182648E+02                    6                  749
    7    3.109426E+07         9.377032E-11    1.133588E-06    3.

The solver found a KKT point. The size of the working set increased to 584 first, and then slowly decreased to 483. According to our previous analysis, this phenomenon is reasonable because this QP is not overly constrained.